In [1]:
import numpy as np
import pandas as pd

In [3]:
df = pd.read_csv(r"diminos_data.csv")
df.head()

,order_id,order_placed_at,order_delivered_at
0,1523111,2023-03-01 00:00:59,2023-03-01 00:18:07.443132
1,1523112,2023-03-01 00:03:59,2023-03-01 00:19:34.925241
2,1523113,2023-03-01 00:07:22,2023-03-01 00:22:28.291385
3,1523114,2023-03-01 00:07:47,2023-03-01 00:46:19.019399
4,1523115,2023-03-01 00:09:03,2023-03-01 00:25:13.619056


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15000 entries, 0 to 14999
Data columns (total 3 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   order_id            15000 non-null  int64 
 1   order_placed_at     15000 non-null  object
 2   order_delivered_at  15000 non-null  object
dtypes: int64(1), object(2)
memory usage: 351.7+ KB


In [5]:
# Convert to datetime
df['order_placed_at'] = pd.to_datetime(df['order_placed_at'])
df['order_delivered_at'] = pd.to_datetime(df['order_delivered_at'])

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15000 entries, 0 to 14999
Data columns (total 3 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   order_id            15000 non-null  int64         
 1   order_placed_at     15000 non-null  datetime64[ns]
 2   order_delivered_at  15000 non-null  datetime64[ns]
dtypes: datetime64[ns](2), int64(1)
memory usage: 351.7 KB


In [7]:
# Delivery time in minutes
df['delivery_time_min'] = (
    df['order_delivered_at'] - df['order_placed_at']
).dt.total_seconds() / 60

In [8]:
df.head()

,order_id,order_placed_at,order_delivered_at,delivery_time_min
0,1523111,2023-03-01 00:00:59,2023-03-01 00:18:07.443132,17.140719
1,1523112,2023-03-01 00:03:59,2023-03-01 00:19:34.925241,15.598754
2,1523113,2023-03-01 00:07:22,2023-03-01 00:22:28.291385,15.104856
3,1523114,2023-03-01 00:07:47,2023-03-01 00:46:19.019399,38.533657
4,1523115,2023-03-01 00:09:03,2023-03-01 00:25:13.619056,16.176984


In [9]:
p95_delivery_time = np.percentile(df['delivery_time_min'], 95)
mean_delivery_time = df['delivery_time_min'].mean()
median_delivery_time = df['delivery_time_min'].median()

print("95th Percentile Delivery Time:", round(p95_delivery_time, 2))
print("Average Delivery Time:", round(mean_delivery_time, 2))
print("Median Delivery Time:", round(median_delivery_time, 2))


95th Percentile Delivery Time: 27.26
Average Delivery Time: 20.5
Median Delivery Time: 15.8


In [17]:
# Percentage of Late Deliveries (>31 mins)
late_delivery_pct = (df['delivery_time_min'] > 31).mean() * 100

print("Late Delivery Percentage (>31 mins):", round(late_delivery_pct, 2), "%")


Late Delivery Percentage (>31 mins): 3.71 %


In [12]:
# Hour-wise 95th Percentile
# Extract order hour
df['order_hour'] = df['order_placed_at'].dt.hour

# Hourly 95th percentile
hourly_p95 = df.groupby('order_hour')['delivery_time_min'].quantile(0.95)

hourly_p95


order_hour
0     28.866990
1     32.104821
2     26.845717
3     25.617373
4     26.259762
5     23.924907
6     26.493076
7     27.154475
8     27.207707
9     25.709020
10    26.759252
11    33.407016
12    24.327900
13    29.714826
14    27.891228
15    24.933851
16    32.438371
17    28.260021
18    24.104592
19    26.801610
20    28.237978
21    28.932870
22    24.611743
23    26.943743
Name: delivery_time_min, dtype: float64

In [14]:
# Identify Risky Hours (95th Percentile > 31 mins)
risky_hours = hourly_p95[hourly_p95 > 31]

risky_hours


order_hour
1     32.104821
11    33.407016
16    32.438371
Name: delivery_time_min, dtype: float64

In [15]:
# Early Warning Metric (Orders Crossing 25 mins)
warning_pct = (df['delivery_time_min'] > 25).mean() * 100

print("Orders crossing 25 mins (early warning):", round(warning_pct, 2), "%")


Orders crossing 25 mins (early warning): 6.07 %


In [16]:
summary = pd.DataFrame({
    'Metric': [
        'Average Delivery Time',
        'Median Delivery Time',
        '95th Percentile Delivery Time',
        'Late Delivery % (>31 mins)',
        'Early Warning % (>25 mins)'
    ],
    'Value': [
        round(mean_delivery_time, 2),
        round(median_delivery_time, 2),
        round(p95_delivery_time, 2),
        round(late_delivery_pct, 2),
        round(warning_pct, 2)
    ]
})

summary


,Metric,Value
0,Average Delivery Time,20.50
1,Median Delivery Time,15.80
2,95th Percentile Delivery Time,27.26
3,Late Delivery % (>31 mins),3.71
4,Early Warning % (>25 mins),6.07


### “I analyzed delivery SLA using 95th percentile, identified late-night risk hours, and suggested proactive monitoring using a 25-minute early warning threshold.”

### Key Metric (What Diminos cares about)
- `95th Percentile Delivery Time = ~27.3 minutes`
- `Good news: Diminos requirement = 95th percentile < 31 minutes`
- `Kanav’s store = 27.3 minutes`
- `The store is currently SAFE and meeting the franchise metric`

### Overall Delivery Performance
- `Average delivery time: ~20.5 minutes`
- `Median delivery time: ~15.8 minutes`
- `Orders delivered after 31 minutes: ~3.7%`
- `This means: Most customers get pizza well within time`
- `Only a small tail of late deliveries is hurting the percentile`

### Risk Areas 
- `When we look hour-wise (order placed time):`
- `Some late-night / early-morning hours (around 1 AM) have 95th percentile > 31 minutes`
- `These hours are pulling the risk upward, even though overall performance is good`
- `This is where Kanav can lose the metric in future if demand increases.`

### Business Insights for Kanav
#### What is working well
- `Normal hours (day + evening) are highly efficient`
- `Kitchen + delivery flow is generally stable`
- `Majority of customers are happy`
#### What needs attention
- `Low-staff or fatigue hours (late night)`
- `Possibly:`
- `Fewer delivery partners`
- `Slower preparation`
- `Longer rider wait times`

### Recommended Actions 
#### Short-term 
- `Add 1 extra delivery rider during late-night risky hours`
- `Pre-prepare popular items after midnight`
- `Track orders >25 mins instead of waiting till 31 mins`

#### Medium-term
- `Create an alert when delivery time crosses 25 mins`
- `Incentivize riders for on-time late-night deliveries`

#### Metric Protection Strategy
- `Target internal 95th percentile ≤ 27 minutes`
- `This gives a safety buffer if order volume increases.`

### Final Conclusion 
- `Kanav’s store is currently meeting Diminos’ 95th percentile SLA, but late-night operations are the biggest risk. Small operational improvements in those hours will secure the franchise long-term.`